In [1]:
import logging
from gensim.models import word2vec



import sys
import os

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
        sys.path.append(module_path)
        
from src import data_prepare

import gzip
from gensim.models import word2vec
import logging

import pandas as pd
from bs4 import BeautifulSoup
import re
from nltk.corpus import stopwords

import warnings
warnings.filterwarnings('ignore')


from time import time  
from collections import defaultdict

import spacy

C:\ProgramData\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s',level=logging.INFO)

post, thread=data_prepare.load_train_data()
#post_test, thread_test=data_prepare.load_test_data()
label_map=data_prepare.load_label_map()

In [3]:
train_data_to_clean=[]
test_data_to_clean=[]

train_data_to_clean=data_prepare.get_all_text_data_from_posts(post,thread)
#test_data_to_clean=data_prepare.get_all_text_data_from_posts(post_test,thread_test)

In [5]:
nlp = spacy.load('en', disable=['ner', 'parser']) # disabling Named Entity Recognition for speed

In [ ]:
def cleaning(doc):
    txt = [token.lemma_ for token in doc if not token.is_stop]
    if len(txt) > 2:
        return ' '.join(txt)
brief_cleaning = (re.sub("[^A-Za-z']+", ' ', str(row)).lower() for row in train_data_to_clean)

txt = [cleaning(doc) for doc in nlp.pipe(brief_cleaning, batch_size=5000, n_threads=-1)]

In [ ]:
from gensim.models.phrases import Phrases, Phraser
sent = [row.split() for row in txt]
phrases = Phrases(sent, min_count=30, progress_per=10000)
sentences = bigram[sent]

In [ ]:
word_freq = defaultdict(int)
for sent in sentences:
    for i in sent:
        word_freq[i] += 1
len(word_freq)

sorted(word_freq, key=word_freq.get, reverse=True)[:10]

In [ ]:
import multiprocessing

from gensim.models import Word2Vec

In [ ]:
w2v_model = Word2Vec(min_count=20,
                     window=2,
                     size=300,
                     sample=6e-5, 
                     alpha=0.03, 
                     min_alpha=0.0007, 
                     negative=20,
                     workers=cores-1)


w2v_model.build_vocab(sentences, progress_per=10000)


w2v_model.train(sentences, total_examples=w2v_model.corpus_count, epochs=30, report_delay=1)

In [ ]:
w2v_model.init_sims(replace=True)